In [1]:
from sklearn import tree
import pandas as pd
import os

In [2]:
df = pd.read_csv("../data_sets/coercing_data/16_columns_coerced.csv")

df.head()

,Unnamed: 0,IMDB,Metacritic,Rotten Tomatoes,Rank,Worldwide Box Office,Awards,Actors_1,Actors_2,Actors_3,directors_1,directors_2,Country_1,language_1,language_2,Genre_1,Genre_2
0,0,8.9,94,0.94,726.0,212928762,1,5771,232,3537,1352,0,63,27,92,5,7
1,1,6.6,53,0.52,107.0,708996336,0,293,1811,2535,1041,0,63,27,0,0,1
2,2,9.3,80,0.91,3935.0,28307092,0,5770,4343,668,511,0,63,27,0,7,0
3,3,8.5,87,0.96,301.0,385524784,1,4175,1095,3561,1457,0,63,27,0,1,4
4,4,7.9,62,0.88,2857.0,45875171,0,1786,4637,402,843,0,63,27,0,4,7


In [3]:
df = df.fillna(0)
df.head()
df['Awards'].min()

0

In [4]:
# Changing award values from 'number of awards won' to 'whether or not it won an award'
df['Awards'].values[df['Awards'] > 0] = 1

df = df.drop(["Unnamed: 0"], axis=1)

# Removing some percentage of non-Oscar winners
df = df.drop(df.query('Awards == 0').sample(frac=.4).index)

# Define the target as whether or not a movie won an award
target = df["Awards"]

In [5]:
data = df.drop("Awards", axis=1)
feature_names = data.columns
data.head()

,IMDB,Metacritic,Rotten Tomatoes,Rank,Worldwide Box Office,Actors_1,Actors_2,Actors_3,directors_1,directors_2,Country_1,language_1,language_2,Genre_1,Genre_2
0,8.9,94,0.94,726.0,212928762,5771,232,3537,1352,0,63,27,92,5,7
1,6.6,53,0.52,107.0,708996336,293,1811,2535,1041,0,63,27,0,0,1
2,9.3,80,0.91,3935.0,28307092,5770,4343,668,511,0,63,27,0,7,0
3,8.5,87,0.96,301.0,385524784,4175,1095,3561,1457,0,63,27,0,1,4
4,7.9,62,0.88,2857.0,45875171,1786,4637,402,843,0,63,27,0,4,7


In [6]:
# Split the data using train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
# Create a Decision Tree Classifier
clf = tree.DecisionTreeClassifier()

In [8]:
# Fit the classifier to the data
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
# Calculate the accuracy for the test data
clf.score(X_test, y_test)

0.8737541528239202

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

labled_weights = pd.DataFrame(sorted(zip(rf.feature_importances_, feature_names), reverse=True))
labled_weights.rename(columns={0:"Weight", 1: "Name"})

0.8986710963455149


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,Weight,Name
0,0.183498,Metacritic
1,0.118390,Rank
2,0.115352,Rotten Tomatoes
3,0.114411,IMDB
4,0.106178,Worldwide Box Office
5,0.067320,directors_1
6,0.063063,Actors_3
7,0.052342,Actors_1
8,0.051564,Actors_2
9,0.036870,Genre_2


In [35]:
from sklearn.metrics import confusion_matrix
average_precision = confusion_matrix(y_test, clf.predict(X_test))

random_forest_matrix = pd.DataFrame(average_precision)
random_forest_matrix.rename(columns={0:"Predicted NO", 1: "Predicted YES"})
actuals = ['Actual YES','Actual NO']
random_forest_matrix[''] = actuals

random_forest_matrix = random_forest_matrix.set_index('')
random_forest_matrix = random_forest_matrix.rename(columns={0:"Predicted NO", 1: "Predicted YES"})

random_forest_matrix

,Predicted NO,Predicted YES
,,
Actual YES,495,45
Actual NO,31,31
